In [5]:
import os
import torch
import numpy as np
import epitran
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    AutoModelForMaskedLM,
)
import kagglehub
from functools import lru_cache

In [6]:
# path = kagglehub.dataset_download("jamiewelsh2/rap-lyrics")

# print("Path to dataset files:", path)

Path to dataset files: /home/toure215/.cache/kagglehub/datasets/jamiewelsh2/rap-lyrics/versions/1


In [28]:
# pd_data_frame = pd.read_csv(
#     "/home/toure215/Documents/BERT_phonetic/DATASETS/rap/updated_rappers.csv"
# )
# pd_data_frame.head(20)

,Unnamed: 0,artist,song,lyric,next lyric
0,0,Fetty Wap,Trap Queen,rgf productions,remy boyz yahah
1,0,Fetty Wap,Trap Queen,remy boyz yahah,1738 ayy
2,0,Fetty Wap,Trap Queen,1738 ayy,im like hey whats up hello
3,0,Fetty Wap,Trap Queen,im like hey whats up hello,seen yo pretty ass soon as you came in the door
4,0,Fetty Wap,Trap Queen,seen yo pretty ass soon as you came in the door,i just wanna chill got a sack for us to roll
5,0,Fetty Wap,Trap Queen,i just wanna chill got a sack for us to roll,married to the money introduced her to my stove
6,0,Fetty Wap,Trap Queen,married to the money introduced her to my stove,showed her how to whip it now she remixin for low
7,0,Fetty Wap,Trap Queen,showed her how to whip it now she remixin for low,she my trap queen let her hit the bando
8,0,Fetty Wap,Trap Queen,she my trap queen let her hit the bando,we be countin up watch how far them bands go
9,0,Fetty Wap,Trap Queen,we be countin up watch how far them bands go,we just set a goal talkin matchin lambos


In [29]:
# print(len(pd_data_frame))
# print(pd_data_frame["song"].nunique())

1445301
17288


In [30]:
# ds = load_dataset("Cropinky/rap_lyrics_english")
# ds

Repo card metadata block was not found. Setting CardData to empty.


Resolving data files:   0%|          | 0/47 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1181216
    })
})

In [31]:
# print(ds["train"][:39])
# ds["train"] = ds["train"][40:]

{'text': ['Lil pump', 'Lil wayne', 'Lil Durk', 'Lil B', 'Lil Uzi Vert', 'Lil Baby', 'Lil Reese', 'Lil Boosie', 'The Notorious B.I.G. ', 'Big Pun', 'Big L', 'Nas', '50 Cent', 'Prodigy', 'Action Bronson', 'Ill Bill', 'Wu-Tang Clan', 'Raekwon', 'Ghostface Killah', 'RZA', 'GZA', "Ol' Dirty Bastard", 'Method Man', 'Inspectah Deck', 'U-God', 'Masta Killa', 'Cappadonna', 'Kanye West', 'Lauryn Hill', 'Jean Grae', 'Lil Kim', 'Missy Elliot', 'Rah Digga', 'MC Lyte', 'Remy Ma', 'Missy Elliott', '', '', 'Foxy Brown']}


In [34]:
ds["train"] = Dataset.from_dict(ds["train"])

In [37]:
# ds = ds.filter(
#     lambda x: "[" not in x["text"] or "]" not in x["text"], num_proc=15
# ).filter(lambda x: x["text"] != "", num_proc=15)
# ds

Filter (num_proc=15):   0%|          | 0/1181176 [00:00<?, ? examples/s]

Filter (num_proc=15):   0%|          | 0/1088460 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1015214
    })
})

In [38]:
# ds["train"][:10]

{'text': ['Ooh, I like the way she move',
  'Shorty my baby, my everything, she the truth',
  "Together we cool, me and her can't lose",
  "Keep 'em on their feet, baby, I know they so confused",
  'Shorty my Beyoncé',
  'Durk and DeJ, Durk and DeJ, Durk and DeJ',
  'Shorty my Beyoncé',
  'Durk and DeJ, Durk and DeJ, Durk and DeJ',
  'My Beyoncé',
  "Trippin' on that drank, but I know she worth it"]}

In [40]:
# pd_ds = ds["train"].to_pandas()
# pd_ds.head(10)

,text
0,"Ooh, I like the way she move"
1,"Shorty my baby, my everything, she the truth"
2,"Together we cool, me and her can't lose"
3,"Keep 'em on their feet, baby, I know they so c..."
4,Shorty my Beyoncé
5,"Durk and DeJ, Durk and DeJ, Durk and DeJ"
6,Shorty my Beyoncé
7,"Durk and DeJ, Durk and DeJ, Durk and DeJ"
8,My Beyoncé
9,"Trippin' on that drank, but I know she worth it"


In [2]:
# epi = epitran.Epitran("eng-Latn")


# @lru_cache(maxsize=None)
# def xsampa_list(word):
#     return epi.xsampa_list(word)


# def is_rhyming(word1, word2):
#     sound1 = xsampa_list(word1)
#     sound2 = xsampa_list(word2)
#     if len(sound1) < 2 or len(sound2) < 2:
#         return False
#     return sound1[-2:] == sound2[-2:]


# # Pre-compute phonetic endings for all verses
# def get_last_phonetic(word):
#     phonemes = xsampa_list(word)
#     return phonemes[-2:] if len(phonemes) >= 2 else phonemes

In [43]:
# pd_ds["last_word"] = pd_ds["text"].apply(lambda x: x.split()[-1])
# pd_ds["phonetic_ending"] = pd_ds["last_word"].apply(get_last_phonetic)
# pd_ds.head()

,text,last_word,phonetic_ending
0,"Ooh, I like the way she move",move,"[u, v]"
1,"Shorty my baby, my everything, she the truth",truth,"[u, T]"
2,"Together we cool, me and her can't lose",lose,"[u, z]"
3,"Keep 'em on their feet, baby, I know they so c...",confused,"[z, d]"
4,Shorty my Beyoncé,Beyoncé,"[k, e]"


In [44]:
# # Convert phonetic_ending lists to tuples for hashing
# pd_ds["phonetic_ending"] = pd_ds["phonetic_ending"].apply(tuple)

# # Group verses by their phonetic endings for quick access to rhyming pairs
# rhyme_groups = (
#     pd_ds.groupby("phonetic_ending").apply(lambda x: x.index.tolist()).to_dict()
# )

/tmp/ipykernel_938215/840613071.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pd_ds.groupby("phonetic_ending").apply(lambda x: x.index.tolist()).to_dict()


In [45]:
phonetic_endings = list(rhyme_groups.keys())

In [46]:
# # Build the dataset
# import random

# rap_ds = pd.DataFrame(columns=["id", "sentence1", "sentence2", "label"])

# for i in range(0, len(pd_ds), 2):
#     last = len(rap_ds)
#     word1 = pd_ds.iloc[i]["last_word"]
#     phonetic1 = pd_ds.iloc[i]["phonetic_ending"]

#     # Find a rhyming pair
#     rhyming_indices = rhyme_groups.get(phonetic1, [])
#     rhyming_idx = i  # Default to self if no other rhyme is found
#     for idx in rhyming_indices:
#         if idx != i:
#             rhyming_idx = idx
#             break

#     rap_ds.loc[last] = [
#         last,
#         pd_ds.iloc[i]["text"],
#         pd_ds.iloc[rhyming_idx]["text"],
#         1,  # Label for rhyming
#     ]

#     # Find a non-rhyming pair by selecting from different phonetic endings
#     non_rhyme_phonetic = phonetic1
#     while non_rhyme_phonetic == phonetic1:
#         non_rhyme_phonetic = random.choice(phonetic_endings)
#     non_rhyme_idx = np.random.choice(rhyme_groups[non_rhyme_phonetic])

#     rap_ds.loc[last + 1] = [
#         last + 1,
#         pd_ds.iloc[i]["text"],
#         pd_ds.iloc[non_rhyme_idx]["text"],
#         0,  # Label for non-rhyming
#     ]

# print("Final row count in rap_ds:", len(rap_ds))

Final row count in rap_ds: 1015214


In [47]:
# from sklearn.model_selection import train_test_split

# train, test = train_test_split(rap_ds, test_size=0.1, random_state=42)
# train, val = train_test_split(train, test_size=0.1, random_state=42)

# train = Dataset.from_pandas(train)
# val = Dataset.from_pandas(val)
# test = Dataset.from_pandas(test)

# rap_ds_hf = DatasetDict({"train": train, "validation": val, "test": test})
# rap_ds_hf.save_to_disk("/home/toure215/Documents/BERT_phonetic/DATASETS/rap/rap_ds_hf")

Saving the dataset (0/1 shards):   0%|          | 0/822322 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/91370 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/101522 [00:00<?, ? examples/s]

In [7]:
rap_ds_hf = load_from_disk(
    "/home/toure215/Documents/BERT_phonetic/DATASETS/rap/rap_ds_hf"
)

rap_ds_rhyme = rap_ds_hf.filter(lambda x: x["label"] == 1, num_proc=os.cpu_count() - 1)

Filter (num_proc=15):   0%|          | 0/822322 [00:00<?, ? examples/s]

Filter (num_proc=15):   0%|          | 0/91370 [00:00<?, ? examples/s]

Filter (num_proc=15):   0%|          | 0/101522 [00:00<?, ? examples/s]

In [8]:
rap_ds_rhyme = rap_ds_rhyme.remove_columns(["__index_level_0__", "id"])


def add_rhyme_label(example):
    label = example["sentence1"].split()[-1]
    return {
        "sentence1": example["sentence1"],
        "sentence2": example["sentence2"],
        "label": label,
    }


rap_ds_rhyme = rap_ds_rhyme.map(add_rhyme_label, num_proc=os.cpu_count() - 1)
rap_ds_rhyme

Map (num_proc=15):   0%|          | 0/411009 [00:00<?, ? examples/s]

Map (num_proc=15):   0%|          | 0/45717 [00:00<?, ? examples/s]

Map (num_proc=15):   0%|          | 0/50881 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 411009
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 45717
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 50881
    })
})

In [9]:
is_phonetic = False

model_path = [
    "bert-base-uncased",
    "psktoure/BERT_BPE_phonetic_wikitext-103-raw-v1",
    "psktoure/BERT_WordPiece_wikitext-103-raw-v1",
]

if is_phonetic:
    model = AutoModelForMaskedLM.from_pretrained(model_path[1])
    tokenizer = AutoTokenizer.from_pretrained(model_path[1])
else:
    model = AutoModelForMaskedLM.from_pretrained(model_path[0])
    tokenizer = AutoTokenizer.from_pretrained(model_path[0])

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [10]:
def translate_sentence(sentence: str) -> str:
    words = sentence.split()
    for i in range(len(words)):
        words[i] = "".join(xsampa_list(words[i]))
    return " ".join(words)


def translate_function(examples):
    examples["sentence1"] = [
        translate_sentence(sentence) for sentence in examples["sentence1"]
    ]
    examples["sentence2"] = [
        translate_sentence(sentence) for sentence in examples["sentence2"]
    ]
    examples["label"] = ["".join(xsampa_list(word)) for word in examples["label"]]
    return examples


if is_phonetic:
    rap_ds_rhyme = rap_ds_rhyme.map(
        translate_function, num_proc=os.cpu_count() - 1, batched=True
    )

In [11]:
import torch
from transformers import PreTrainedTokenizerBase


class CustomDataCollator:
    def __init__(self, tokenizer: PreTrainedTokenizerBase, padding=True, max_length=64):
        self.tokenizer = tokenizer
        self.mask_token_id = tokenizer.mask_token_id
        self.padding = padding
        self.max_length = max_length

    def __call__(self, examples):

        sentence1 = [example["sentence1"] for example in examples]
        sentence2 = [example["sentence2"] for example in examples]
        targets = [example["label"] for example in examples]

        encoded_targets = self.tokenizer(targets, add_special_tokens=False)

        batch = self.tokenizer(
            sentence1,
            sentence2,
            padding=self.padding,
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )

        input_ids = batch["input_ids"]
        labels = input_ids.clone()

        for i, idx in enumerate(input_ids):
            sep_token_indices = torch.where(idx == self.tokenizer.sep_token_id)[0]
            start = sep_token_indices[0] - len(encoded_targets[i])
            end = sep_token_indices[0]
            input_ids[i, start:end] = self.mask_token_id
            labels[i, :start] = -100
            labels[i, end:] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": batch["attention_mask"],
            "labels": labels,
        }

In [12]:
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100, reduction="mean")

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [13]:
data_collator = CustomDataCollator(tokenizer)

training_args = TrainingArguments(
    output_dir="/tmp/fine_tuned_bert",
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_strategy="no",
    remove_unused_columns=False,
    fp16=True,
)

# Initialize the Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=rap_ds_rhyme["train"],
    eval_dataset=rap_ds_rhyme["validation"],
    data_collator=data_collator,
)

/home/toure215/miniconda3/envs/bert/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [14]:
trainer.train()

  0%|          | 0/38535 [00:00<?, ?it/s]

  0%|          | 0/1429 [00:00<?, ?it/s]

{'eval_loss': 1.3175690174102783, 'eval_runtime': 22.1577, 'eval_samples_per_second': 2063.258, 'eval_steps_per_second': 64.492, 'epoch': 1.0}


  0%|          | 0/1429 [00:00<?, ?it/s]

{'eval_loss': 1.0568487644195557, 'eval_runtime': 21.2686, 'eval_samples_per_second': 2149.505, 'eval_steps_per_second': 67.188, 'epoch': 2.0}


  0%|          | 0/1429 [00:00<?, ?it/s]

{'eval_loss': 0.9762381315231323, 'eval_runtime': 21.2673, 'eval_samples_per_second': 2149.633, 'eval_steps_per_second': 67.192, 'epoch': 3.0}
{'train_runtime': 1753.6891, 'train_samples_per_second': 703.105, 'train_steps_per_second': 21.974, 'train_loss': 1.1264493723238613, 'epoch': 3.0}


TrainOutput(global_step=38535, training_loss=1.1264493723238613, metrics={'train_runtime': 1753.6891, 'train_samples_per_second': 703.105, 'train_steps_per_second': 21.974, 'total_flos': 3.05655543996498e+16, 'train_loss': 1.1264493723238613, 'epoch': 3.0})

In [15]:
def evaluate_rhyme_indices(model, dataset, tokenizer, k=5):

    model = model.to("cuda")
    model.eval()
    res = []
    batch_size = 256

    for i in range(0, len(dataset), batch_size):
        print(f"Processing batch {i}/{len(dataset)}...", end="\r")
        batch = dataset[i : i + batch_size]
        batch_sequence = [
            {key: batch[key][j] for key in batch}
            for j in range(len(batch["sentence1"]))
        ]
        inputs = data_collator(batch_sequence)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            labels = inputs["labels"]

        count = 0

        for j in range(len(batch["sentence1"])):
            # Identify the position of the masked token
            masked_token_index = (
                inputs["input_ids"][j] == tokenizer.mask_token_id
            ).nonzero(as_tuple=True)[0]

            # Get the target index
            # target_index = inputs["labels"][j, masked_token_index]

            targets = labels[j, masked_token_index]
            # Get the top-k predictions
            top_k_indices = logits[j, masked_token_index].topk(k).indices.squeeze(0)
            if i < 16 and j < 8:
                print("targets:", targets, "-- top_k_indices:", top_k_indices)

            # Check if the target index is in the top-k predictions
            ok = True
            for idx, target in enumerate(targets):
                if target not in top_k_indices[idx]:
                    ok = False
            if ok:
                count += 1

        # Append accuracy for this batch
        res.append(count / len(batch["sentence1"]))

    # Return the mean Top-k Accuracy
    return {"score": np.mean(res)}

In [16]:
evaluate_rhyme_indices(model, rap_ds_rhyme["test"], tokenizer, k=5)

targets: g batch 0/50881...tensor([2009, 1007], device='cuda:0') -- top_k_indices: tensor([[2009, 4485, 2008, 2718, 2023],
        [1007, 1065,  999, 1012, 1033]], device='cuda:0')
targets: tensor([2126, 1007], device='cuda:0') -- top_k_indices: tensor([[ 2126,  2360, 17812,  4931,  3100],
        [ 1007,  1012,  1065,  1029,  1011]], device='cuda:0')
targets: tensor([3231], device='cuda:0') -- top_k_indices: tensor([3231, 2190, 3891, 2717, 2695], device='cuda:0')
targets: tensor([6237], device='cuda:0') -- top_k_indices: tensor([ 2606,  2073, 16200,  6771,  2045], device='cuda:0')
targets: tensor([1026, 8945, 2015, 1028], device='cuda:0') -- top_k_indices: tensor([[ 1026,  9152,  1028,  1000,  1006],
        [ 8945, 23033,  5811,  2015,  1012],
        [ 2015,  4757,  3401,   999, 15094],
        [ 1028,   999,  1000,  1065,  1007]], device='cuda:0')
targets: tensor([2033], device='cuda:0') -- top_k_indices: tensor([ 2033, 24369, 22712,  6838,  5261], device='cuda:0')
targets: tensor(

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.86 GiB. GPU 0 has a total capacity of 8.00 GiB of which 132.56 MiB is free. Including non-PyTorch memory, this process has 6.83 GiB memory in use. Of the allocated memory 5.18 GiB is allocated by PyTorch, and 1.15 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)